In [2]:
import torch 
import os 
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%cd ../

/workspace/Walk_Video_PyTorch/project


In [3]:
from models.pytorchvideo_models import WalkVideoClassificationLightningModule
from pytorch_lightning import Trainer

/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [4]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 8
opt.batch_size = 64
opt.gpu_num = 0
opt.model = "resnet"
opt.model_depth = 50


In [5]:
model = WalkVideoClassificationLightningModule(opt)
# model = model.load_from_checkpoint("/workspace/Walk_Video_PyTorch/logs/resnet/0603/checkpoints/epoch=99-step=1800.ckpt")
model.eval()

checkpoint = torch.load("/workspace/Walk_Video_PyTorch/logs/resnet/0719_resnet_depth50/checkpoints/epoch=60-step=6832.ckpt")

model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [6]:
from dataloader.data_loader import WalkDataModule
from pytorch_lightning import loggers as pl_loggers 

# load test dataset 
module = WalkDataModule(opt)
module.setup()
test_data = module.test_dataloader()

# for the tensorboard
tb_logger = pl_loggers.TensorBoardLogger(save_dir="/workspace/Walk_Video_PyTorch/project/tests/logs", name=opt.model, version=opt.version)


trainer = Trainer(
    accelerator="auto",
    devices=1,
    gpus=opt.gpu_num,
    logger=tb_logger,
    max_epochs=-1,
)

trainer.test(dataloaders=module, model=model)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

/workspace/Walk_Video_PyTorch/project/models/pytorchvideo_models.py:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(batch["label"])
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 44. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 43. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_acc_step       │    0.4514285624027252     │
│      test_dice_step       │    0.4514285624027252     │
│         test_loss         │     2.116831064224243     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.116831064224243,
  'test_acc_step': 0.4514285624027252,
  'test_dice_step': 0.4514285624027252}]

In [ ]:
input_data = next(iter(test_data))


In [ ]:
input_data['video'].shape

input_data["video_name"]

In [ ]:
input_data['label']

In [ ]:
preds = model(input_data['video'])

In [ ]:
post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=1).indices

In [ ]:
pred_classes

In [ ]:
classname = {}

classname[0] = 'asd'
classname[1] = 'asd_not'

In [ ]:
real_calss = []

for i in input_data['label'].tolist():
    real_calss.append(classname[i])

In [ ]:
real_calss

In [ ]:
# pred_class_names = []
# for num in range(opt._BATCH_SIZE):
#     for i in pred_classes[i]:
#         pred_class_names.append(classname[int(i)])


pred_class_names = [classname[int(i)] for i in pred_classes]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print("real label: %s" % ",".join(real_calss))

In [ ]:
pred_class_names == real_calss

In [ ]:
result = []

for i in range(len(real_calss)):
    if pred_class_names[i] == real_calss[i]:
        result.append("true")
    else:
        result.append("false")

result